In [4]:
# http://cnrpark.it/
# importing all the packages need to build a Convolutional neural network (CNN) model
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

import requests #For HTTP Post
import os

batch_size = 32             # number of samples or examples the algorithm takes at a time
learning_rate = 0.0001      # a parameter that is tuned ti optimize the algorithm
epoch = 5                   # # number of times over all the dataset

In [5]:
# mount google drive to access data in folder
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
## Image dimensions are 113 X 150 pixels
print('Number of Images in Park_train: ',len(os.listdir("/content/drive/My Drive/parkingData/park_train")))
print('Number of Images in Park_test: ',len(os.listdir("/content/drive/My Drive/parkingData/park_test")))

Number of Images in Park_train:  5711
Number of Images in Park_test:  1161


In [9]:
#Initiate a model
model = Sequential()

# extraction the learnable features and reducing dimension 
model.add(Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(113,150,3)))
model.add(MaxPool2D(pool_size=(2, 2), strides=(1, 1)))

model.add(Conv2D(32,3,3,activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64,3,3,activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

# Preparing the images for input in ANN by converting to single column
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

#Image Augmentation - Processing and tranforming the image to generate more images
from keras.preprocessing.image import ImageDataGenerator

datagenerator_train = ImageDataGenerator(rescale=1./255,
                                        shear_range=0.2,
                                         zoom_range=0.2,
                                         horizontal_flip = True,
                                         validation_split=0.2)

datagenerator_test = ImageDataGenerator(rescale=1./255)

# training the dataset
train_dataset = datagenerator_train.flow_from_directory("/content/drive/My Drive/parkingData",
                                                        target_size = (113, 150),
                                                        batch_size = batch_size,
                                                        class_mode = 'binary',
                                                       subset='training')

test_dataset = datagenerator_train.flow_from_directory("/content/drive/My Drive/parkingData",
                                                        target_size = (113, 150),
                                                        batch_size = batch_size,
                                                        class_mode = 'binary',
                                                      subset='validation')

# Fit the model
model.fit(train_dataset,
          batch_size = batch_size,
          #steps_per_epoch = len(train_dataset) // batch_size,
          epochs = 5,
          verbose = 1,
          validation_data = test_dataset)
          #validation_steps = 1374)   


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 111, 148, 32)      896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 110, 147, 32)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 36, 49, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 18, 24, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 6, 8, 64)          18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 3, 4, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 768)              

In [15]:
import numpy as np
from keras.preprocessing import image
import random

parkData = '' #String to append to POST
parkUrl = 'http://parkdata.us.to/parkdata.php' #Link to web server
myobj = {'parkdata': parkData} #HTTP Post parameters

# To classify new image
pic = "/content/drive/My Drive/parkingData/park_test/20150708_0830_26.jpg"
new_img = image.load_img(pic,target_size = (113, 150))

#Get random image from file

path = r"/content/drive/My Drive/parkingData/park_train"
random_filename = random.choice([
   x for x in os.listdir(path)
   if os.path.isfile(os.path.join(path, x))])
#print(random_filename)

#new_img = image.load_img(random_filename,target_size = (113, 150))
#Process the image
new_img = image.img_to_array(new_img)
new_img = np.expand_dims(new_img, axis=0)

#Predict new image
result = model.predict(new_img)
print(result)    # Predicts the new image as 1 occupied and 0 free

train_dataset.class_indices
if result[0][0] == 1:
  output = 'It is occupied!'
  parkData += '1\n'
  
else:
  output = 'It is available!'
  parkData += '0\n'

print(output)
x = requests.post(parkUrl, data = myobj)

20150703_1430_38.jpg
[[1.]]
It is occupied!
